In [1]:
using Rocket
using Distributions
using ReactiveMP
using BenchmarkTools

import Base: show

┌ Info: Precompiling Rocket [df971d30-c9d6-4b37-b8ff-e965b2cb3a40]
└ @ Base loading.jl:1273
┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1273
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.
┌ Warning: Package ForneyLab does not have Distributions in its dependencies:
│ - If you have ForneyLab checked out for development and have
│   added Distributions as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ForneyLab
└ Loading Distributions into ForneyLab from project dependen

In [2]:
function createSubgraph(index::Int)
    noise_add      = AdditionNode("[$index] noise_add");
    noise_node     = GaussianMeanVarianceNode("[$index] noise_node");
    noise_mean     = ConstantVariable("[$index] noise_mean", 0.0, noise_node.mean);
    noise_variance = ConstantVariable("[$index] noise_variance", 1.0, noise_node.variance);
    
    tmp = RandomVariable("[$index] tmp", noise_node.value, noise_add.in2)
    yn  = ObservedVariable("[$index] yn", noise_add.out)
    
    return (yn, noise_add)
end

function createGraph(size::Int)
    gmv_0   = GaussianMeanVarianceNode("[0] gmv");
    gmv_0_m = PriorVariable("[0] gmv_m", gmv_0.mean);
    gmv_0_v = PriorVariable("[0] gmv_v", gmv_0.variance);
    
    add_c_0 = AdditionNode("[0] add_c");
    x0      = RandomVariable("[0] x", gmv_0.value, add_c_0.in1)
    c0      = ConstantVariable("[0] c", 1.0, add_c_0.in2)
    
    index = 1
    
    prev_add_c = add_c_0
    
    xs = Vector{RandomVariable}(undef, size)
    ys = Vector{ObservedVariable}(undef, size)
    
    while index < size
        equality_n    = EqualityIOONode("[$index] equality")
        yn, noise_add = createSubgraph(index)
        
        xn  = RandomVariable("[$index] xn", prev_add_c.out, equality_n.in1)
        xn_ = RandomVariable("[$index] xn_", equality_n.out1, noise_add.in1)
        
        add_c = AdditionNode("[$index] add_c");
        
        xn__ = RandomVariable("[$index] xn__", equality_n.out2, add_c.in1)
        cn   = ConstantVariable("[$index] cn", 1.0, add_c.in2)
        
        xs[index] = xn
        ys[index] = yn
        
        prev_add_c = add_c
        
        index += 1
    end
    
    last_noise_add = AdditionNode("[last] noise_add");
    x_last         = RandomVariable("[last] x", prev_add_c.out, last_noise_add.in1)
    
    last_noise_node     = GaussianMeanVarianceNode("[last] noise_node");
    last_noise_mean     = ConstantVariable("[last] noise_mean", 0.0, last_noise_node.mean);
    last_noise_variance = ConstantVariable("[last] noise_variance", 200.0, last_noise_node.variance);
    
    z      = RandomVariable("[last] z", last_noise_node.value, last_noise_add.in2)
    y_last = ObservedVariable("[last] y", last_noise_add.out)
    
    xs[size] = x_last
    ys[size] = y_last
    
    return (xs, ys, gmv_0_m, gmv_0_v)
end

createGraph (generic function with 1 method)

In [3]:
function smoothing_replay()
    N = 600
    data = collect(1:N) + sqrt(200.0) * randn(N);
    
    xs, ys, prior_m, prior_v = createGraph(N);
    
    messages = Vector{AbstractMessage}(undef, N)
    
    subscribe!(inference(xs[1]) |> take(1), lambda(on_next = (d) -> messages[1] = d))
    subscribe!(inference(xs[N]) |> take(1), lambda(on_next = (d) -> messages[N] = d))
    
    update!(prior_m, 0.0)
    update!(prior_v, 1000.0)
    for i in 1:N
       update!(ys[i], data[i])
    end
    
    for i in 2:N-1
        subscribe!(inference(xs[i]) |> take(1), lambda(on_next = (d) -> messages[i] = d))
    end
    
    return messages
end

smoothing_replay (generic function with 1 method)

In [5]:
@btime smoothing_replay();

  3.200 s (23255858 allocations: 726.22 MiB)


In [4]:
@time smoothing_replay()

  7.263871 seconds (32.17 M allocations: 1.161 GiB, 5.72% gc time)


600-element Array{AbstractMessage,1}:
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=1.0795530070819974, σ=0.04085868768828826)) 
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=2.079553007081998, σ=0.04085868768828826))  
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=3.0795530070819974, σ=0.04085868768828825)) 
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=4.079553007081997, σ=0.04085868768828825))  
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=5.079553007081996, σ=0.04085868768828826))  
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=6.079553007081998, σ=0.04085868768828826))  
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=7.079553007081997, σ=0.04085868768828826))  
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=8.079553007081998, σ=0.04085868768828826))  
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=9.079553007082, σ=0.04085868768828826))     
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=10.079553007082, 

In [9]:
@btime smoothing_replay();

  2.477 s (21404287 allocations: 642.40 MiB)


In [8]:
@time smoothing_replay()

  3.428928 seconds (23.78 M allocations: 770.039 MiB, 4.84% gc time)


600-element Array{AbstractMessage,1}:
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=1.175828971929451, σ=0.04085868768828826))  
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=2.1758289719294512, σ=0.04085868768828826)) 
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=3.1758289719294504, σ=0.04085868768828825)) 
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=4.17582897192945, σ=0.04085868768828825))   
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=5.175828971929452, σ=0.04085868768828826))  
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=6.175828971929452, σ=0.04085868768828826))  
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=7.175828971929452, σ=0.04085868768828826))  
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=8.175828971929453, σ=0.04085868768828826))  
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=9.175828971929452, σ=0.04085868768828826))  
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=10.17582897192945